### Iris-Dataset

  - Regressão linear

In [ ]:
import os, sys
import pandas as pd # pandas e seu alias pd
import numpy as np  # numpy  e seu alias np

from   scipy import stats
#-- for ANOVA
import statsmodels.api as sm
from   statsmodels.formula.api import ols

from scipy import stats
is_colab = False

if is_colab:
    # create src and upload stat_lib into it
    from src import stat_lib
else:
    sys.path.insert(1, '../src/')
    import stat_lib

import seaborn as sns

import matplotlib.pyplot as plt # matplotlib e seu alias plt
# %matplotlib inline

### Abrir Iris dataset

In [ ]:
os.listdir("../data/")

In [ ]:
fname = "../data/Iris.csv"
if os.path.exists(fname):
    df = pd.read_csv(fname)
    print(df.shape, fname)
else:
    print("Impossivel abrir '%s'"%(fname))

### Tabela da Iris de Ronald Fisher & Edgar Anderson (1936)

https://en.wikipedia.org/wiki/Iris_flower_data_set

### Partes da flor

![sepal and petal](../figure/flower-parts.jpg)

https://www.britannica.com/science/sepal

![sepal and petal](../figure/iris.png)

![sepal and petal](../figure/iris_len_width.jpeg)

In [ ]:
print(df.shape)
df.head(3)

In [ ]:
df.columns = ['id', 'sepal_len', 'sepal_width', 'petal_len', 'petal_width', 'species']
df.tail(3)

In [ ]:
df.species.unique()

In [ ]:
df.columns

In [ ]:
name = 'Iris-setosa'
df_seto = df[df.species == name]
df_seto.shape

### Exercício: mostre por groupby que há 50 amostras por espécie

   df.groupby .... variavel .. count()

### Matriz de correlação - dim = num variáveis ** 2

In [ ]:
df_seto[ ['sepal_len', 'sepal_width', 'petal_len', 'petal_width'] ].corr()

### corr(x, y) == corr(y, x)

### Histograma e Scatter plot entre sépala e pétalas de Iris (orquídea)

In [ ]:
ret = pd.plotting.scatter_matrix(df_seto[ ['sepal_len', 'sepal_width', 'petal_len', 'petal_width'] ], figsize=(8, 8))
# plt.title('%s pair correlations'%(name));

In [ ]:
ret

### Matriz de correlação na forma de Heatmap

In [ ]:
corr_mat = df_seto[ ['sepal_len', 'sepal_width', 'petal_len', 'petal_width'] ].corr()
sns.heatmap(corr_mat, annot = True)
plt.title('%s correlation matrix'%(name));

### Exercício: fazer a matriz de correlação das outras 2 espécies

### As medidas (comprimento (len) and largura (width)) devem se aproximar de uma distribuição normal
  - mostre que esta afirmação é verdadeira

In [ ]:
ret, text, text_stat, stat, pvalue = stat_lib.calc_normalidade_SWT(df_seto.sepal_len)
text, text_stat

In [ ]:
ret, text, text_stat, stat, pvalue = stat_lib.calc_normalidade_SWT(df_seto.sepal_width)
text, text_stat

In [ ]:
ret, text, text_stat, stat, pvalue = stat_lib.calc_normalidade_SWT(df_seto.petal_len)
text, text_stat

In [ ]:
ret, text, text_stat, stat, pvalue = stat_lib.calc_normalidade_SWT(df_seto.petal_width)
text, text_stat

### Como as distribuições são normais
### Teste de Hipótese (t-test) entre comparimentos de pétalas/sépalas entre espécies

### Vamos calcular correlação de Pearson entre sepal_len x sepal_width
  - a correlação de Pearson é somente entre 2 seguem a distribuição normal (contínua)
  - não vamos ver aqui, mas há a correlação de Speerman quando uma ou as duas variávies não segue uma distribuição normal mas podem ser ordenadas

referências:
  - https://www.statsmodels.org/stable/regression.html
  - https://realpython.com/numpy-scipy-pandas-correlation-python/
  - https://machinelearningmastery.com/how-to-use-correlation-to-understand-the-relationship-between-variables/
  - https://benalexkeen.com/correlation-in-python/
  

### O que é regressão linear:
  - uma regressão linear é calculada através de duas variáveis
    - x = variável independente
    - y = variável dependente
      - dependente: se correlação for boa ou ótima
      - independente: se correlação não existir, for fraca
    
### A correlação mede o quanto uma variável dependente (y) depende (se correlaciona) com uma variável independete (x)    
    
  - se y depende 100% de x, então a correlação = 1 
    - exemplo y = 3 * x
  - se y depende 100% de x, mas inversamente, então a correlação = -1
    - ou seja, quando x cresce y decresce ou vice-versa
    - exemplo y = -3 * x
  - se y não depende de x, então a correlação fica próximo a 0
    - se duas variáveis são independentes (temperatura, T, da atmosfera e altura das crianças numa escola, h) então dizemos que não há correlação entre T e h
    - -0.2 < corr < 0.2 - correlação muito fraca - pode ter sindo encontrada 'ao acaso'
  - logo a correlação varia de -1 a +1, passando pelo zero.
    
### Faixa de correlação
  - Na prática 'se diz' que
    - [.8 a 1] = ótima correlação (pode ser negativo, também)
    - [.6 a .8[ = correlação boa
    - [.4 a .6[ = correlação regular
    - [.2 a .4[ = correlação ruim
    - [0 a .2[ = correlação muito ruim, sem correlação
    
### Importante! Na Biologia a maioria dos fenômenos são NÃO LINEARES
  - devido: complexidade
  - devido: fenômenos emergentes
    - quando a soma de todos os efetios << que o fenômeno final observado == EMERGÊNCIA
    
  Y = x * z * t**2 * sqrt(k) + 5

### Abaixo, o scattrer plot entre largura e comprimento das sépalas da Iris Setosa

  - não vemos ainda a regressão linear que pode ser calcula   
  - a regressão linear é uma reta que passa entre os pontos minimizando o erro
  - erro = desvio entre o valor Y e seu respectivo ponto na reta f(x), onde f(x) = y, reta predita


In [ ]:
plt.figure(figsize=(8,4))
plt.scatter(df_seto.sepal_width, df_seto.sepal_len)
plt.xlabel('sepal length (cm)')
plt.ylabel('sepal width (cm)')
plt.title('%s scatter plot'%(name));

### A correlação de Pearson só pode ser calculada se os valores forem oriundos de distribuições normais

In [ ]:
from scipy.stats import pearsonr

cor, pval = pearsonr(df_seto.sepal_width, df_seto.sepal_len)
'correlação = %.3f and p-value = %.2e'%(cor, pval)

### O p-value é retirado da estatística F de variância within e between

In [ ]:
#-- ols = ordinary least square
#              largura ~ (depende) de comprimento
model = ols('sepal_width ~ sepal_len', data=df_seto)
fit = model.fit()
aov_table = sm.stats.anova_lm(fit, typ=2)
aov_table

In [ ]:
# fit.summary()

In [ ]:
fit.pvalues

### Regressão Linear

  - uma reta pode ser descrita da forma:

y =  intercept + a * x  
(lembre-se y = ax+b)

  - ou mais generalizadamente
  
$y = b_0 + b_1 * x$

onde b0 = intercept = coeficiente linear  
e    b1 = a         = coeficiente angular
  

### O fitting calcula os parâmetro 'intercept' e 'a'

In [ ]:
intercept, a = fit.params
intercept, a

### Agora podemos plotar a regressão
  - como já afirmamos, acima: uma regressão linear é um cáculo matemático para se calcular uma reta que passa por entre todos valores (x,y) apresentados, minizando o erro total (distância da reta até cada valor y), ora denominado de resíduo

In [ ]:
xseq = np.linspace(4.2, 6, 22)
# fitting --> reta predita y = ax + b = ax + intercepto
yseq = intercept + a * xseq

cor, pval = pearsonr(df_seto.sepal_width, df_seto.sepal_len)
stri = 'correlação = %.3f and p-value = %.2e'%(cor, pval)

# reta vermelha --> yhat - preditor que minimiza erros
# erros : desvio de um ponto qq (y) de seu valor em yhat
# erro: diferença entre valor real - valor estimado (predito)
plt.figure(figsize=(8,4))
plt.scatter(df_seto.sepal_len, df_seto.sepal_width)
plt.plot(xseq, yseq, color='red')
plt.xlabel('sepal length (cm)')
plt.ylabel('sepal width (cm)')
plt.title('regressão entre Largura e Comprimento da sépala da %s\n%s'%(name, stri));

### Resíduos

### É a diferença entre cada valor de Y (valores observados) pelo Yhat (valores preditos)

![residuals](../figure/residuals.png)

### Gráfico de resíduos

In [ ]:
plt.figure(figsize=(8,4))

plt.scatter(df_seto.sepal_len, fit.resid, color='red')
plt.plot(xseq, [0]*len(xseq), color='black')

plt.xlabel('sepal length (cm)')
plt.ylabel('residuals (errors) = y - yhat')
plt.title('residual plot');

### Soma dos resíduos

In [ ]:
np.sum(fit.resid)

### O que boa regressão linear?
  - É aquela com cor > 0.6, pvalue muito baixo (<< 0.05)
  - soma dos resíduos tenda a zero
  - histograma dos resíduos seja uma distribuição Normal

### Distribuição dos resíduos

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

residuos = fit.resid
color = 'red'

ax = sns.histplot(residuos, stat='density', color=color, alpha=0.3, ax=ax)
sns.rugplot(residuos, color=color, alpha=0.3, ax=ax)

plt.title('residual distribution (similar to normal?)');

In [ ]:
ret, text, text_stat, stat, pvalue = stat_lib.calc_normalidade_SWT(fit.resid)
text, text_stat

### Não pode haver correlação nos resíduos para uma perfeita regressão linear

In [ ]:
cor, pval = pearsonr(fit.resid, df_seto.sepal_len)
'correlation = %.3f and p-value = %.2e'%(cor, pval)

### Exercício: simule valores para correlação cujo valor se aproxima de +1, -1, e 0 (zero)

#### Recomendação:
  - nas aulas anteriores aprendemos a gerar números randômicos

    a) se x = random.normal(2, 1) e y = 2 * x, então a correlação = +1  
    b) se x = random.normal(2, 1) e y = random.normal(3, .9), então a correlação é ??  
    c) se x = random.normal(2, 1) e y = -1 * x + random.normal(0, 1), então a correlação é ??  
    <br>
      - neste último caso random.normal(0, 1) é denominado de erro aleatório ou ruído branco

### Exemplo não linear

y = 2 * x**2 + 1

In [ ]:
xseq = np.linspace(-1, 4, 26)
yseq = 2 * xseq**2 + 1

plt.figure(figsize=(8,4))
plt.plot(xseq, yseq, color='blue')
plt.xlabel('x')
plt.ylabel('y = 2x**2 + 1')
plt.title('Equação de segundo grau')

In [ ]:
cor, pval = pearsonr(yseq, xseq)
'correlation = %.3f and p-value = %.2e'%(cor, pval)

In [ ]:
df2 = pd.DataFrame([xseq, yseq]).T
df2.columns = ['x', 'y']
df2.head()

In [ ]:
model = ols('y ~ x', data=df2)
fit = model.fit()
aov_table = sm.stats.anova_lm(fit, typ=2)
aov_table

In [ ]:
fit.pvalues

In [ ]:
intercept, a = fit.params
intercept, a

### Predição 

In [ ]:
xseq = np.linspace(-1, 4, 26)
yseq = 2 * xseq**2 + 1
yhat = a*xseq + intercept

plt.figure(figsize=(8,4))
plt.plot(xseq, yseq, color='blue')
plt.plot(xseq, yhat, color='red')
plt.xlabel('x')
plt.ylabel('y = 2x**2 + 1')

stri = 'correlation = %.3f and p-value = %.2e'%(cor, pval)
plt.title('Equação de segundo grau\n%s'%(stri));

In [ ]:
np.sum(fit.resid)

### Gráfico de resíduos

In [ ]:
plt.figure(figsize=(8,4))

plt.scatter(df2.x, fit.resid, color='red')
plt.plot(xseq, [0]*len(xseq), color='black')

plt.xlabel('x')
plt.ylabel('residuals (errors) = y - yhat')
plt.title('residual plot');

In [ ]:
sns.distplot(fit.resid, kde=True, rug=True, norm_hist=True, color='red', rug_kws={"color": 'red', "alpha": .1,})
plt.title('residual distribution (similar to normal?)');